In [1]:
import pandas as pd
import numpy as np

from wikiapi import WikiWrapper

from tqdm import tqdm
import multiprocessing as mp
from multiprocessing import Pool

from p_tqdm import p_map

from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [1]:
q_hist = (
    """
    PREFIX dbo: <http://dbpedia.org/ontology/>
    SELECT DISTINCT ?Event_1
    WHERE { ?Event_1 a dbo:Event .
            ?Event_1 a dbo:MilitaryConflict . }

    LIMIT 1000
    """
)

q_nonhist_1 = (
    """
    PREFIX dbo: <http://dbpedia.org/ontology/>
    SELECT DISTINCT ?Artist_1
    WHERE { ?Artist_1 a dbo:Artist . }

    LIMIT 500
    """
)

q_nonhist_2 = (
    """
    PREFIX dbo: <http://dbpedia.org/ontology/>
    SELECT DISTINCT ?Animal_1
    WHERE { ?Animal_1 a dbo:Animal . }

    LIMIT 500
    """
)

In [3]:
hist_events_name = WikiWrapper.download_pages_name(q_hist)
non_hist_events_name = WikiWrapper.download_pages_name(q_nonhist_1)
non_hist_events_name = non_hist_events_name + WikiWrapper.download_pages_name(q_nonhist_2)

In [18]:
df = pd.DataFrame(hist_events_name, columns=["Name"])
df["Abstract"] = ""
df["Label"] = 1

In [19]:
df1 = pd.DataFrame(non_hist_events_name, columns=["Name"])
df1["Abstract"] = ""
df1["Label"] = 0

In [20]:
df = pd.concat([df, df1])
df.reset_index(inplace=True, drop=True)
df = df.sample(frac=1).reset_index(drop=True)

In [21]:
df 

,Name,Abstract,Label
0,Bataan_Death_March,,1
1,Hikitia,,0
2,Walleye,,0
3,Bill_Parker_(comics),,0
4,Anju_Panta,,0
...,...,...,...
1995,Operation_Coronado_II,,1
1996,Azov_campaigns_(1695–96),,1
1997,Battle_of_Jwa-won,,1
1998,"Action_of_April_17,_1837",,1


In [22]:
df["Abstract"] = df.parallel_apply(WikiWrapper.get_extract, axis=1)

In [4]:
df.dropna(subset=["Abstract"], inplace=True)
df.reset_index(inplace=True, drop=True)

In [29]:
df["Abstract"] = df["Abstract"].parallel_apply(lambda x: x.replace(",", "").replace("|", ""))

In [30]:
df

,Name,Abstract,Label
0,Bataan_Death_March,The Bataan Death March (Filipino: Martsa ng Ka...,1
1,Hikitia,Hikitia is a working self-propelled floating s...,0
2,Walleye,The walleye (Sander vitreus synonym Stizosted...,0
3,Bill_Parker_(comics),William Lee Parker (September 11 1911 – Januar...,0
4,Anju_Panta,Anju Panta (Nepali: अन्जु पन्त) is a Nepalese ...,0
...,...,...,...
1995,Operation_Coronado_II,Operation Coronado II was the second of eleven...,1
1996,Azov_campaigns_(1695–96),,1
1997,Battle_of_Jwa-won,The Battle of Jwa-won (Hangul: 좌원전투 Hanja: 坐原戰...,1
1998,"Action_of_April_17,_1837",The Battle of the Brazos River was an engageme...,1


In [6]:
df.to_csv("data/wiki/wiki.csv", index=False, sep="|")